## Term Project Proposal : Chrome_History를 비교, 검색어 관련 미검색 url 추출 

<h2>(1) 주제 선정 이유 : </h2><br><br>
    사람들은 다른 사용자가 어떤 정보를 가지고 있는지, 그 정보를 가지고 어떻게 해결하였는지 궁금해하며, 그렇기 때문에 우리는 <strong>서로의 정보를 공유하는 집단 지성의 사회</strong>를 살고 있다.<br><br>
    자신이 검색한 검색어와 관련된 url 추천은 많다. google의 연관검색어 시스템이 잘 되어있다.<br><br> 하지만, 자신이 검색했던 검색어에 대해 자신이 중요한 사이트지만,<br><strong> 자신이 발견하지 못한 사이트.</strong>
    <strong>남들은 많이 봤는데 자신만이 놓치고 있는 정보를 얻는</strong>, 그런 사이트를 추천해주는 편리한 프로그램을 구현한 페이지가 필요하다고 생각하였다.<br><br>
    그 중 사용자들이 제일 많이 쓰고 있는 검색엔진은 <h3>구글의 크롬이다.</h3><br>
    많은 사용자들이 존재하기 때문에, 이를 오픈소스로 공개한다면. 많은 이용자들이 편리함을 얻을 것이라 생각하여
    크롬에 관해서 국한지었지만, 이는 나중에 다른 검색엔진에도 충분히 적용이 될 수 있다고 생각한다. <br>
    
<h2>(2) 데이터 획득 : </h2><br><br>
    C:\Users\user_name\AppData\Local\Google\Chrome\User Data\Default\History 접근하여 자신이 크롬으로 검색했던 기록이 DB 형태로 존재한다.<br>
    user는 로컬 컴퓨터에서 History 파일을 눌러 upload 버튼을 누르면 scp로 서버에 전달된다.<br>
    History Upload 되자마자 history_2_pd_db.py 실행되면서 History_{username}.db로 생성된다.
    
<h2>(3) 분석을 위한 데이터의 가공: </h2><br>
    <strong>history_2_pd_db.py : </strong><br>
    pandas, sqlite3 모듈을 사용하여 .db파일을 열어 dataframe화 시키고, <br>
    이를 토대로 사용자가 사이트를 접속할 때마다 History 파일을 보내는데, 이를 업데이트 하는 파일로 정의하였다.<br>
    또한, 이 파일은 History의 내용을 누적한다. 누적하는 과정은 History에서 last_visit_time이라는 column이 존재한다. 따라서 누적된 파일의 제일 최근 visit_time 이후 시간부터 History의 부분만 추출해서 History_{username}.db에 누적하는 방식을 사용하고 있다.<br>
    <strong>user_history_merge.py :</strong> <br>
    여러 user들의 History_{username}.db 으로 저장되어있는 .db파일을 하나로 합치는 과정에서 사용자들의 url이 겹치는 것을 중요하게 판단 (그 url을 몇 명이 들어가보았는지가 이 사이트가 중요한지의 척도), 이를 user_count로 정의해서, merge하며 겹칠 때 마다 하나씩 올린다. 이의 data들을 History_all_users.db로 저장한다.<br>
    <strong>token_word_judge.py : </strong><br>
    .db에서 title 부분을 다루는 함수들을 정의하였다.<br>
    search(dirname) : 실행되는 파일의 dir에서 dir만 출력해주는 함수<br>
    db_sentence_2_token_list(database_history_all_users_data_list) : 긴 sentence를 RegexTokenizer 모듈을 이용하여 token으로 나눠서 title에 있던 자리에 다시 담는 함수.<br>
    token_judge_en_lower_ko_noun(token) : RegexTokenizer로 나눈 token을 받으면 영어, 한글을 파악해서 영어면, lower로 , 한글이면 noun으로 return 해주는 함수 <br>
    kor_or_eng_judge(token_word) : token_word를 받은 것을 아스키코드를 이용하여, 영어라면 'en', 한글이라면 'ko', 이외의 문자라면 0을 return하는 함수. 이는 google_translator 쓰기 위해 문자가 영어인지 한글인지를 비교해준다. <br>
    google_translator(input_word_list) : google 번역기의 모듈을 사용해서, 영어를 한글로, 한글을 영어로 바꿔서, 원래 문자와 번역한 문자를 같은 list에 넣는 함수. 이는 파이썬 == python 을 같은 의미로 두기 위한 작업이다. <br>
    word_bisect(a, x, lo=0, hi=None) : word에 관한 이진 검색이다. 기존 이진 검색 함수 같은 경우에는, 단어나 숫자가 없다면 그 근방의 index를 추출한다. 이를 보완하기 위해 만약 단어가 있다면 index를 return, 단어가 없다면 None을 return 하는 함수이다. <br>
    <strong>title_token.py : </strong><br>
    user들의 History를 모은 History_all_users.db 파일에서 column = title을 token화<br>
    이를 token 기준으로 sort해서 History_all_users_title_token.db 로 저장한다.<br>
    
<h2>(4) 결과 도출 : </h2><br><br>
    keyword_tokenize.py : <br>
    웹페이지에서 사용자가 keyword를 입력하면, 위에서 token을 나눈 기준과 동일한 방법으로 token으로 나누어준다. <br> 
    token들을 History_all_users_title_token.db에서 검색한다. <br> 
    이를 검색한 user의 History_{username}.db 파일에서 url과 겹치는 부분을 제외한 나머지 url을 추린다.<br> 
    이를 사용자가 입력한 keyword의 token들이 token에 얼마나 겹치는지,<br> 그리고 user_count, visit_count 순으로 우선순위를 설정. 순위대로 url을 보여준다.<br>
    만약 위의 조건을 만족하지 못한다면, keyword 입력한 google 검색창의 url을 보여준다.<br>
    
<h2>(5) 결론 : </h2><br><br>
    우리는 keyword를 입력한 후, 자신의 검색기록을 다른 user들과 비교하면서,<br> 
    자신이 검색한 검색어에 대해 찾아보지 못했던, url을 추출받으면서,<br>
    집단지성으로 도출되는 효과를 이 웹페이지를 통해 볼 수 있다.<br>
    또한 자신이 실수로 체크하지 못했던 url을 보여줌으로써 검색어에 대한 정보를 보충할 수 있다.<br>
    
<h2>(6) 참고문헌: 위의 작업을 위하여 획득한 자료, 인터넷 정보, 논문, 도서 등을 나열함 </h2><br><br>
    Pandas : https://pandas.pydata.org/<br>
    nunmpy : https://numpy.org <br>
    konlpy : https://konlpy-ko.readthedocs.io/ko/v0.4.3/<br>
    soynlp : https://github.com/lovit/soynlp<br>
    sqlite3 : https://docs.python.org/ko/3/library/sqlite3.html<br>
    googletrans : https://pypi.org/project/googletrans/<br>
    
    
    
<h2>(7) 별첨: (3)의 획득한 데이터 원본</h2><br><br>
    History <br>
    History_{username}.db<br>
    History_all_users.db<br>
    History_all_users_title_token.db <br>

<h2>(8) 역할 : </h2><br><br>
    이름 : 유혁준, 학과 : 응용화학과, 역할 : History 파일 분석, keyword 자연어처리<br>
    이름 : 강형구 (비수강생), 학과 : 컴퓨터공학과, 역할 : 웹페이지 관리<br>
     
<h2>(9) 별첨: (3)을 (4)로 변환하는 등의 작업을 위하여 본인이 직접 개발한 Python 소스코드 원본</h2><br><br>
    crawling_history 안에 있는 모든 .py 파일입니다.<br>
    
    
<h2>(10) 추가내용(웹페이지 시스템)</h2> : 
    <a href> http://w500history.duckdns.org:8080/ </a>이번에 만든 홈페이지입니다. 현재 AWS Lightsail 에 서버를 구축해놓은 상황이며, 보안을 위해 duckdns에서 DNS를 받아 현재 url을 가지고 있습니다. <br>
    <img src = "image_1.png"><br>
    처음 들어가시면 이러한 홈페이지가 나옵니다. Intro에는 홈페이지의 목적, 어떻게 해야하는지에 대한 정보가 나와있습니다.<br>
    <img src = "image_2.png"><br>
    로그인 하는 방식은 google Oauth를 사용했습니다. 본인의 구글계정으로 로그인을 합니다.<br>
    <img src = "image_3.png"><br>
    다음과 같은 화면이 나온다면, OS에 따라서 History 파일을 경로 복사한 후, choose 버튼을 누른 후,History 파일을 찾아서 upload 버튼을 누릅니다.<br> 
    <img src = "image_4.png"><br>
    다음과 같은 화면이 실행된다면, keyword를 입력한 후 엔터를 누릅니다.
    <img src = "image_5.png"><br>
    결과 화면입니다.<br>